[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/unboxai/examples-gallery/blob/main/text-classification/demo-banking.ipynb)


# Text classification using a custom

This notebook illustrates how custom models can be upladed to the Unbox platform. In this particular case, we train two fastText models on different datasets and then combine them to do inference.

## Importing the modules

In [1]:
import os
import fasttext
import numpy as np

## Training two fastText models on two datasets

We have stored the datasets on the following S3 bucket. If, for some reason, you get an error with the `curl`, feel free to copy and paste the URL in your browser and download the txt file. 

In [2]:
!curl -0 https://unbox-static-assets.s3.us-west-2.amazonaws.com/examples-datasets/text-classification/rec.txt -o rec.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13.3M  100 13.3M    0     0  2034k      0  0:00:06  0:00:06 --:--:-- 2714k


In [3]:
!curl -0 https://unbox-static-assets.s3.us-west-2.amazonaws.com/examples-datasets/text-classification/rec_type.txt -o rec_type.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1952k  100 1952k    0     0   840k      0  0:00:02  0:00:02 --:--:--  842k


In [11]:
rec_model = fasttext.train_supervised(input="rec.txt", lr=0.3, epoch=100, loss='hs')

Read 2M words
Number of words:  211125
Number of labels: 2
Progress: 100.0% words/sec/thread: 6906188 lr:  0.000000 avg.loss:  0.396676 ETA:   0h 0m 0s


In [15]:
rec_type_model = fasttext.train_supervised(input="rec_type.txt", lr=0.3, epoch=100, loss='hs')

Read 0M words
Number of words:  49236
Number of labels: 4
Progress: 100.0% words/sec/thread: 6616957 lr:  0.000000 avg.loss:  1.092409 ETA:   0h 0m 0s


Let's inspect how each model predicts the same sentence:

In [4]:
rec_model.predict(["science and politics"]), rec_type_model.predict(["science and politics"])

(([['__label__not.rec']], [array([0.914432], dtype=float32)]),
 ([['__label__rec.autos']], [array([0.61953795], dtype=float32)]))

 The predictions are different, mainly because on the dataset `rec.txt`, there are only 2 classes represented, namely `not.rec` and `rec`. On the other hand, on the `rec_type.txt` dataset, there are 4 classes: `rec.sport.hockey`, `rec.motorcycles`, `rec.sport.baseball`, and `rec.autos`. Let's just confirm that:

In [5]:
rec_class_names = [s.replace("__label__", "") for s in rec_model.labels]
rec_type_class_names = [s.replace("__label__", "") for s in rec_type_model.labels]
rec_k = len(rec_class_names)
rec_type_k = len(rec_type_class_names)

In [6]:
print(f"Number of classes on the `rec.txt` dataset: {rec_k}")
print(f"Number of classes on the `rec_type.txt` dataset: {rec_type_k}")

Number of classes on the `rec.txt` dataset: 2
Number of classes on the `rec_type.txt` dataset: 4


In [7]:
print(f"Classes represented on the `rec.txt` dataset: {rec_class_names}")
print(f"Classes represented on the `rec_type.txt` dataset: {rec_type_class_names}")

Classes represented on the `rec.txt` dataset: ['not.rec', 'rec']
Classes represented on the `rec_type.txt` dataset: ['rec.sport.hockey', 'rec.motorcycles', 'rec.sport.baseball', 'rec.autos']


## Defining how to combine the two model's predictions for inference

For inference, we want to use a combination of each model's prediction. Let's define the function `get_predictions`, which, for a given text sample, combines the predictions of both models:

In [8]:
all_class_names = ['not.rec', 'rec.sport.hockey', 'rec.motorcycles', 'rec.sport.baseball', 'rec.autos']

In [9]:
def get_predictions(models, text_list):
    
    rec_model, rec_type_model = models
    
    predictions = rec_model.predict(text_list, k=rec_k)
    x, y = predictions
    
    predictions = rec_type_model.predict(text_list, k=rec_k)
    x2, y2 = predictions
    
    probabilities_full_list = []
    for rec_label_list, rec_prob_list, rec_type_label_list, rec_type_prob_list in zip(x, y, x2, y2):
        rec_label_prob_pair_dict = {}
        rec_type_label_prob_pair_dict = {}
        
        for rec_lbl, rec_prob, rec_type_lbl, rec_type_prob in zip(
            rec_label_list, rec_prob_list, rec_type_label_list, rec_type_prob_list
        ):
            rec_label_prob_pair_dict[
                rec_lbl.replace("__label__", "")
            ] = rec_prob
            rec_type_label_prob_pair_dict[
                rec_type_lbl.replace("__label__", "")
            ] = rec_type_prob
        
        rec_weight = rec_label_prob_pair_dict['rec']
        not_rec_weight = rec_label_prob_pair_dict['not.rec']
        probabilities_list = []
        
        if rec_weight > not_rec_weight:
            for cls in all_class_names:
                if cls in rec_type_label_prob_pair_dict:
                    p = rec_type_label_prob_pair_dict[cls]
                    probabilities_list.append(p)
                else:
                    probabilities_list.append(0.0)
        
        if not_rec_weight > rec_weight:
            for cls in all_class_names:
                if cls == 'not.rec':
                    p = rec_label_prob_pair_dict[cls]
                    probabilities_list.append(p)
                elif cls in rec_type_label_prob_pair_dict:
                    p = rec_type_label_prob_pair_dict[cls]*rec_weight
                    probabilities_list.append(p)
                else:
                    probabilities_list.append(0.0)
                    
        probabilities_full_list.append(probabilities_list)
        
        
    return np.array(probabilities_full_list)

Now, let's see how the model predicts a couple sample sentences:

In [10]:
phrases = ["science", "motorcycles are cool"]

get_predictions((rec_model, rec_type_model), phrases)

array([[0.98981708, 0.00253816, 0.00280518, 0.        , 0.        ],
       [0.        , 0.24129315, 0.        , 0.36813089, 0.        ]])

## Saving the model binaries 

Finally, we can save our trained models for packaging.

In [ ]:
folder_name = "./dependencies"
rec_model_path = f"{folder_name}/rec_model.bin"
rec_type_model_path = f"{folder_name}/rec_type_model.bin"

# Save model binaries for packaging
os.makedirs(folder_name)
rec_model.save_model(rec_model_path)
rec_type_model.save_model(rec_type_model_path)

## Unbox part!

### Instantiating the client

In [ ]:
import unboxapi

client = unboxapi.UnboxClient("YOUR_API_KEY_HERE")

### Creating a project on the platform

In [14]:
project = client.create_project(name="FastText Stacked Model",
                                description="Project to Demo Fasttext Stacked")

Creating project on Unbox! Check out https://unbox.ai/projects to have a look!


### Uploading the model

First, we define the `custom_model_code`, which is the code that needs to be executed before running inference with our combined models. In this case, we need to load the model binaries:

In [15]:
custom_model_code = f"""
import fasttext

rec_model = fasttext.load_model("{rec_model_path}")
rec_type_model = fasttext.load_model("{rec_type_model_path}")
model = (rec_model, rec_type_model)
"""

Now, we are ready to upload it to Unbox as a custom model:

In [16]:
from unboxapi.models import ModelType
from unboxapi.tasks import TaskType

model = project.add_model(
    function=get_predictions, 
    model=None,
    dependent_dir=folder_name,
    custom_model_code=custom_model_code,
    model_type=ModelType.custom,
    task_type=TaskType.TextClassification,
    class_names=all_class_names,
    requirements_txt_file="./requirements.txt",
    name='Stacked model',
    description='there are two models here'
)

Bundling model and artifacts...
Uploading model to Unbox! Check out https://unbox.ai/models to have a look!
